In [1]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame


In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


In [5]:
// определяем структуру
val schema  =  new StructType()
  .add("target", "int", true)
  .add("id", "long", true)
  .add("raw_timestamp", "string", true)
  .add("query_status", "string", true)
  .add("author", "string", true)
  .add("tweet", "string", true)


// читаем данные и преобразовываем к 0\1
val data = spark.read.format("csv")
    .schema(schema)
    .load("/home/jovyan/data/training.1600000.processed.noemoticon.csv")
    .selectExpr("(case when target=4 then 1 else 0 end) as label","tweet")

schema = StructType(StructField(target,IntegerType,true), StructField(id,LongType,true), StructField(raw_timestamp,StringType,true), StructField(query_status,StringType,true), StructField(author,StringType,true), StructField(tweet,StringType,true))
data = [label: int, tweet: string]


[label: int, tweet: string]

In [8]:
// разбиваем датасет на тест и трейн (30% held out for testing).
val Array(trainingData, testData) = data.randomSplit(Array(0.07, 0.03), seed=42)

trainingData.groupBy("label").count().show()
testData.groupBy("label").count().show()

+-----+------+
|label| count|
+-----+------+
|    1|559742|
|    0|560492|
+-----+------+

+-----+------+
|label| count|
+-----+------+
|    1|240258|
|    0|239508|
+-----+------+



trainingData = [label: int, tweet: string]
testData = [label: int, tweet: string]


[label: int, tweet: string]

+-----+------+
|label| count|
+-----+------+
|    1|559742|
|    0|560492|
+-----+------+

+-----+------+
|label| count|
+-----+------+
|    1|240258|
|    0|239508|
+-----+------+



In [3]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath = /home/jovyan/data/events-stream
modelPath = /home/jovyan/models/spark-ml-model
dataSchema = StructType(StructField(tweet,StringType,true))
inputDF = [tweet: string]


[tweet: string]

In [5]:
/*
inputDF.writeStream
      .format("console")
      .outputMode("append")
      .start()             // Start the computation
      .awaitTermination()  // Wait for the computation to terminate
*/

lastException: Throwable = null


Name: Syntax Error.
Message: 
StackTrace: 

In [6]:
inputDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF.show()
}.start()

org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@71401cff

+--------------------+
|               tweet|
+--------------------+
|is feeling a bit ...|
|@jenwithpinktoes ...|
|Ok for real now.....|
|@bigbluebronco  i...|
|Thinkin bout my g...|
|Long, crap day. W...|
|&amp; my Narrativ...|
|Sat in a gay bar ...|
|@Ruth_Z  Just Awe...|
|is a cheesecake m...|
|lying in my bed w...|
|@ahoova I'm putti...|
|@RabihAtallah hal...|
|        new clothes |
|Cant Wait For Ton...|
+--------------------+

+--------------------+
|               tweet|
+--------------------+
|equinux.com just ...|
|sad i don't wanna...|
|Gathering a few t...|
|@AlwaysConvinced ...|
|@MiaLaBelle let i...|
|@brenda_song what...|
|Omg today didn't ...|
|@Louie_Kapoohy di...|
|Oh, I finished th...|
|@joystiq cant dm ...|
|@shutupshayla Why...|
|@thylady yay! wel...|
|Wait i am sorry w...|
|I'll just post bo...|
|nothinggggggggggg...|
|@PaulaAbdul my co...|
|Im going to cruis...|
|@wittertainment I...|
+--------------------+

+--------------------+
|               tweet|
+--------